In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.io as io
import os
from torch.utils.data import Dataset, DataLoader


In [2]:
base_dir = "/projects/dsci410/baseballprediction/pitch_classifier"


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pschale/mlb-pitch-data-20152018")

print("Path to dataset files:", path)

Path to dataset files: /home/nikhils/.cache/kagglehub/datasets/pschale/mlb-pitch-data-20152018/versions/2


In [4]:
pitches = pd.read_csv('pitchdata/pitches.csv')
atbats = pd.read_csv('pitchdata/atbats.csv')
names = pd.read_csv('pitchdata/player_names.csv')

In [5]:
filtered_atbats = atbats[(atbats['batter_id'] == 453286) | (atbats['pitcher_id'] == 453286)]

In [6]:
filtered_atbats.head()

,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
631,2015000632,434158,Walk,201500010,1,0,0,R,453286,L,True
632,2015000633,431151,Pop Out,201500010,1,1,0,R,453286,R,True
633,2015000634,446263,Strikeout,201500010,1,2,0,R,453286,L,True
634,2015000635,150212,Groundout,201500010,1,3,0,R,453286,R,True
640,2015000641,502517,Flyout,201500010,2,1,0,R,453286,L,True


In [7]:
filtered_atbats1 = filtered_atbats[['pitcher_id', 'ab_id', 'g_id']]
filtered_pitches = pitches[['ab_id', 'spin_rate', 'type_confidence', 'pitch_type', 'pitch_num']]

validating = pd.merge(filtered_atbats1, filtered_pitches, on='ab_id', how='inner')
validating.head()

,pitcher_id,ab_id,g_id,spin_rate,type_confidence,pitch_type,pitch_num
0,453286,2015000632,201500010,2704.451,2.0,FF,1.0
1,453286,2015000632,201500010,2536.460,2.0,FF,2.0
2,453286,2015000632,201500010,2546.016,2.0,FF,3.0
3,453286,2015000632,201500010,1628.200,2.0,CH,4.0
4,453286,2015000632,201500010,2040.141,2.0,FF,5.0


In [14]:
games = pd.read_csv('pitchdata/games.csv')
filtered_games = games[['g_id', 'date']]
most_valid = pd.merge(validating, filtered_games, on='g_id', how='inner')
start_date = '2018-01-01'
end_date = '2018-12-31'
most_valid = most_valid.loc[(most_valid['date'] >= start_date) & (most_valid['date'] <= end_date)]
most_valid

,pitcher_id,ab_id,g_id,spin_rate,type_confidence,pitch_type,pitch_num,date
10811,453286,2018001877,201800024,2134.325,2.000,FF,1.0,2018-03-30
10812,453286,2018001877,201800024,2356.836,2.000,FF,2.0,2018-03-30
10813,453286,2018001877,201800024,734.463,2.000,FC,3.0,2018-03-30
10814,453286,2018001877,201800024,2371.196,2.000,FF,4.0,2018-03-30
10815,453286,2018001877,201800024,2037.082,2.000,CH,5.0,2018-03-30
...,...,...,...,...,...,...,...,...
14579,453286,2018179463,201802352,2045.125,0.906,FF,8.0,2018-09-25
14580,453286,2018179463,201802352,1638.230,0.877,FF,9.0,2018-09-25
14581,453286,2018179463,201802352,183.841,0.892,SL,10.0,2018-09-25
14582,453286,2018179464,201802352,2144.930,0.898,FF,1.0,2018-09-25


In [33]:
from torch.utils.data import Dataset
import torchvision.io as io
from torchvision import transforms

class CustomVideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        """
        Args:
            video_dir (string): Directory containing the video files.
            transform (callable, optional): Optional transform to be applied on the video tensor.
        """
        self.video_dir = video_dir
        self.video_files = [
            os.path.join(video_dir, f) for f in os.listdir(video_dir)
            if f.lower().endswith('.avi')
        ]
        print(f"Found {len(self.video_files)} video files in {video_dir}")
        self.transform = transform

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = self.video_files[idx]
        video_tensor, audio_tensor, metadata = io.read_video(video_path, pts_unit='sec')
        
        # might change
        label = 0
        
        if self.transform:
            video_tensor = self.transform(video_tensor)
        
        return video_tensor, label


In [34]:
video_transform = transforms.Compose([
    lambda x: x 
])

#collate function to pad videos in a batch to have the same number of frames by adding zeros so every video length matches the max 
def custom_collate_fn(batch):
    """
    Batch is a list of tuples: (video_tensor, label).
    video_tensor shape: (num_frames, H, W, C) where num_frames may vary.
    """
    videos, labels = zip(*batch)
    
    
    max_frames = max(video.shape[0] for video in videos)
    
    padded_videos = []
    for video in videos:
        num_frames, H, W, C = video.shape
        pad_frames = max_frames - num_frames
        if pad_frames > 0:
           
            padding = torch.zeros((pad_frames, H, W, C), dtype=video.dtype)
            video = torch.cat([video, padding], dim=0)
        padded_videos.append(video)
    
    # Stack padded videos into a single tensor of shape (batch_size, max_frames, H, W, C)
    padded_videos = torch.stack(padded_videos)
    labels = torch.tensor(labels)
    
    return padded_videos, labels

In [35]:
video_dir = 'cvids/'

dataset = CustomVideoDataset(video_dir=video_dir, transform=video_transform)

total_samples = len(dataset)
train_size = int(0.8 * total_samples)
val_size = int(0.1 * total_samples)
test_size = total_samples - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


Found 10 video files in cvids/


In [36]:
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

for batch_videos, batch_labels in train_loader:
    print(f"Batch videos shape: {batch_videos.shape}")  # (batch_size, max_frames, H, W, C)
    break

Batch videos shape: torch.Size([4, 41, 1080, 1920, 3])


In [38]:
from torch.utils.data import DataLoader

batch_size = 4 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)



for batch_videos, batch_labels in train_loader:
    print(f"Batch videos shape: {batch_videos.shape}")
    break


Batch videos shape: torch.Size([4, 41, 1080, 1920, 3])
Batch labels shape: torch.Size([4])
